# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    if dirs:
    # join the file path and roots with the subdirectories using glob
        file_path_list = glob.glob(os.path.join(root,'*'))
        print(len(file_path_list))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
print(file_path_list)
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
#Create a Keyspace 

session.execute("CREATE KEYSPACE IF NOT EXISTS sparkify WITH REPLICATION = {'class':'SimpleStrategy','replication_factor':3}")

#### Set Keyspace

In [ ]:
#Setting KEYSPACE
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [ ]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

session.execute("""CREATE TABLE IF NOT EXISTS music_library_session (sessionId int, itemInSession int,artist text, 
                   firstName text, gender text, lastName text, length float, level text, location text,  song text, 
                   userId int, primary key(sessionId, itemInSession))""")
                    

### Table 1 - music_library which can be queried based on session ID and the item in session. 

#### Data Modeling for this table is based on the Query 1 : Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4. <br/> Here, the partition key would be sessionID  and within each partition, each row would be uniquely identified by iteminsession, thus it can be the clustering column. 

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements 
        query = "INSERT INTO music_library_session (sessionId, itemInSession, artist, firstName, gender, lastName, length, level, location, song, userId)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]),line[0],line[1],line[2],line[4],float(line[5]),line[6],line[7],line[9],int(line[10])))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
rows = session.execute("SELECT artist,song,length FROM music_library_session where sessionId = 338 and iteminsession = 4")

for row in rows:
    print(row)

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [ ]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

session.execute("""CREATE TABLE IF NOT EXISTS music_library_userid (userId int, sessionId int, itemInSession int, 
                    artist text, firstName text, gender text, lastName text, length float, level text, location text,  
                    song text, primary key((userid, sessionid), iteminsession))""")
                    
                    

### Table 2 - music_library which can be queried based on session ID, userid and the item in session. 

#### Data Modeling for this table is based on the Query 2 : Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182. <br/> Here, the partition key would be a combination of user ID and session ID,  and thus the data would be partitioned based on both these columns. Within each partition, however, iteminsession is used to uniquely identify each row. Thus iteminsession would be the clustering column. 

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements 
        query = "INSERT INTO music_library_userid (userId, sessionId, itemInSession, artist, firstName, gender, lastName, length, level, location, song)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]),int(line[8]), int(line[3]),line[0],line[1],line[2],line[4],float(line[5]),line[6],line[7],line[9]))

In [ ]:
rows = session.execute("SELECT artist,song, firstname, lastname FROM music_library_userid where userid = 10 and sessionId = 182")

for row in rows:
    print(row)

In [ ]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

session.execute("""CREATE TABLE IF NOT EXISTS music_library_song ( song text, userId int, firstName text, lastName text, artist text,
                   gender text, itemInSession int,  length float, level text, location text, sessionId int, 
                   primary key(song, userId))""")
                    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements 
        query = "INSERT INTO music_library_song ( song,  userId, firstName, lastName, sessionId, itemInSession, artist,  gender, length, level, location)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (line[9], int(line[10]), line[1],line[4], int(line[8]), int(line[3]),line[0],line[2],float(line[5]),line[6],line[7]))                 
                    

### Table 3 - music_library which can be queried based on song and uniquely identified by a user. 

#### Data Modeling for this table is based on the Query 3 : Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'. <br/> Here, the partition key would be the song, and thus the data would be partitioned based on each song. Within each partition, however, userid is used to uniquely identify each row. Thus userid would be the clustering column. 

In [ ]:
rows = session.execute("SELECT firstname, lastname FROM music_library_song where song = 'All Hands Against His Own' ")

for row in rows:
    print(row)

### Drop the tables before closing out the sessions

In [ ]:
session.execute("DROP TABLE music_library_session")

In [ ]:
session.execute("DROP TABLE music_library_userid")

In [ ]:
session.execute("DROP TABLE music_library_song")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()